# Pynq 1

Pynq1 shall hold RGB light module in PMODA

Pynq1 shall hold PmodAD2 module in PMODB

Pynq1 shall have the heartbeat sensor attached to the PmodAD2 module using channel 1 for signal (v1)

Pynq1 shall be powered via USB

Pynq1 shall have a serial connection with M1 (for ifconfig or other direct pynq management)

Pynq1 shall have an ethernet connection with M1

Pynq1 shall run on "pynq --version==2.7.0"

Pynq1 shall read from ADC using Pynq Pmod ADC library

In [1]:
from pynq.lib import Pmod_ADC

Pynq1 shall maintain the capability to manipulate on-board GPIO, buttons, LEDs, and PMODs using the BaseOverlay library

In [2]:
from pynq.overlays.base import BaseOverlay

In [3]:
baseB = BaseOverlay("base.bit")
adc = Pmod_ADC(baseB.PMODB)

Pynq1 shall use the sleep function to delay execution of instructions for debugging assistance

In [4]:
from time import sleep

Pynq1 shall use the multiprocessing library to execute parallel processes with the help of the OS library to command these processes to specific CPUs.

In [5]:
import multiprocessing
import os

Pynq1 shall use the queue library to gather and hold ADC data

Note: This allows us to push ADC data to the queue and let old data drop off once queue reaches a max count. 

In [6]:
import queue

Pynq1 shall use the socket library for communication to either M1 or Pynq2

In [7]:
import socket

Pynq1 shall host a server process on CPU1 that will perform the following duties:
* Bind to localhost and an arbitrary port number
* Create and connect to a TCP socket
* Wait for a client to initiate contact

Note: Two things can happen now. We can either send the ADC data to be processed elsewhere or process the ADC data on the pynq board itself. For this first iteration, we just want to prove that we can establish a connection and send data across it.

* Send ADC data via socket to the client every "period" seconds

In [18]:
# For debugging purposes
from IPython.display import display, clear_output

def server_process(host,port,period):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((host, port))
    s.listen(1) #  How many allowable connections?
    conn, addr = s.accept()
    print('Connection from:', addr)
    while True:
        clear_output(wait=True)
        adc_value = adc.read(ch1=1,ch2=0,ch3=0) #  Get data from adc
        conn.sendall(str(adc_value[0]).encode())
        display("Sending " + str(adc_value) + " to " + str(addr))
        sleep(period)
    conn.close()

In [19]:
freq = 10 #  Hz
server_process = multiprocessing.Process(target=server_process,
                                         args=('',5002,1/freq))
server_process.start() 
os.system("taskset -pc {} {}".format(1, server_process.pid)) 
server_process.join()
print("Server Process " + server_process.name + " is finished")

Process Process-6:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-18-f9dc2e418937>", line 13, in server_process
    conn.sendall(str(adc_value[0]).encode())
ConnectionResetError: [Errno 104] Connection reset by peer


Server Process Process-6 is finished
